In [14]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np

In [2]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [3]:
model = keras.models.load_model('model.h5')

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 60)        1560      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 60)        90060     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 60)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 30)        16230     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 30)          8130      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 30)          0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 30)          0

In [5]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp7b6anp09/assets


In [6]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

1438028

In [7]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)

In [16]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [29]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

In [30]:
test = cv2.imread('0.png')

In [33]:
img = np.asarray(test)
img = cv2.resize(img, (32, 32))
img = preprocessing(img)
img = np.asarray(img,dtype=np.float32)
img = img.reshape(1, 32, 32, 1)

In [35]:
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], img)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)

Prediction results shape: (1, 4)


In [36]:
prediction_classes = np.argmax(tflite_model_predictions, axis=1)#

In [38]:
prediction_classes[0]

0